In [3]:
import findspark
findspark.init("/opt/spark/")
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import regexp_replace
import configparser

In [4]:
spark = SparkSession.builder \
.appName("Odev_1") \
.config("spark.jars.packages", "org.postgresql:postgresql:42.7.2") \
.master("local[2]") \
.getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/train/.ivy2/cache
The jars for the packages stored in: /home/train/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a2418283-23f1-458c-870a-6b164016c7c2;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.2 in central
	found org.checkerframework#checker-qual;3.42.0 in central
:: resolution report :: resolve 197ms :: artifacts dl 10ms
	:: modules in use:
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.postgresql#postgresql;42.7.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	--------------------------------------------

In [5]:
! java --version


openjdk 11.0.22 2024-01-16 LTS
OpenJDK Runtime Environment (Red_Hat-11.0.22.0.7-1.el7_9) (build 11.0.22+7-LTS)
OpenJDK 64-Bit Server VM (Red_Hat-11.0.22.0.7-1.el7_9) (build 11.0.22+7-LTS, mixed mode, sharing)


In [6]:
#csv download

In [7]:
! wget -O /opt/examples/datasets/dirty_store_transactions.csv \
https://github.com/erkansirin78/datasets/raw/master/dirty_store_transactions.csv

--2024-04-25 18:14:40--  https://github.com/erkansirin78/datasets/raw/master/dirty_store_transactions.csv
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/erkansirin78/datasets/master/dirty_store_transactions.csv [following]
--2024-04-25 18:14:40--  https://raw.githubusercontent.com/erkansirin78/datasets/master/dirty_store_transactions.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2609524 (2.5M) [text/plain]
Saving to: ‘/opt/examples/datasets/dirty_store_transactions.csv’

100%[======================================>] 2,609,524   5.31MB/s   in 0.5s   

2024-04-25 18:14:41 (5.31 MB/s) - ‘/op

In [34]:
#dataset read

df = spark.read.option("header",True) \
.option("inferSchema",True) \
.option("sep",",") \
.csv("file:///opt/examples/datasets/dirty_store_transactions.csv")


In [9]:
df.show()

+--------+--------------+----------------+----------+---+------+--------+------+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|    CP|DISCOUNT|    SP|      Date|
+--------+--------------+----------------+----------+---+------+--------+------+----------+
|  YR7220|     New York(|     Electronics|  12254943|$31|$20.77|   $1.86|$29.14|2019-11-26|
|  YR7220|     New York+|       Furniture| 72619323C|$15| $9.75|    $1.5| $13.5|2019-11-26|
|  YR7220|     New York |     Electronics| 34161682B|$88|$62.48|    $4.4| $83.6|2019-11-26|
|  YR7220|     New York!|         Kitchen|  79411621|$91|$58.24|   $3.64|$87.36|2019-11-26|
|  YR7220|      New York|         Fashion| 39520263T|$85|   $51|   $2.55|$82.45|2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204|$37|$24.05|   $0.74|$36.26|2019-11-26|
|  YR7220|      New York|       Cosmetics| 86610412D|$80| $48.8|    $6.4| $73.6|2019-11-26|
|  YR7220|     New York$|         Kitchen| 52503356^|$71| $42.6|   $5.68|$65.32|

In [10]:
df.printSchema()

root
 |-- STORE_ID: string (nullable = true)
 |-- STORE_LOCATION: string (nullable = true)
 |-- PRODUCT_CATEGORY: string (nullable = true)
 |-- PRODUCT_ID: string (nullable = true)
 |-- MRP: string (nullable = true)
 |-- CP: string (nullable = true)
 |-- DISCOUNT: string (nullable = true)
 |-- SP: string (nullable = true)
 |-- Date: date (nullable = true)



In [11]:
df.dtypes

[('STORE_ID', 'string'),
 ('STORE_LOCATION', 'string'),
 ('PRODUCT_CATEGORY', 'string'),
 ('PRODUCT_ID', 'string'),
 ('MRP', 'string'),
 ('CP', 'string'),
 ('DISCOUNT', 'string'),
 ('SP', 'string'),
 ('Date', 'date')]

In [38]:
#string trim

str_col=[]

for col in df.dtypes:
    if col[1]=="string":
        str_col.append(col[0])
str_col


['STORE_ID',
 'STORE_LOCATION',
 'PRODUCT_CATEGORY',
 'PRODUCT_ID',
 'MRP',
 'CP',
 'DISCOUNT',
 'SP']

In [13]:
df2=df

In [14]:
for col in str_col:
    df2=df2.withColumn(col,F.trim(col))



In [15]:
#trim deneme

df2.filter("PRODUCT_CATEGORY=='Electronics'").limit(5).toPandas()

,STORE_ID,STORE_LOCATION,PRODUCT_CATEGORY,PRODUCT_ID,MRP,CP,DISCOUNT,SP,Date
0,YR7220,New York(,Electronics,12254943,$31,$20.77,$1.86,$29.14,2019-11-26
1,YR7220,New York,Electronics,34161682B,$88,$62.48,$4.4,$83.6,2019-11-26
2,YR7220,New York,Electronics,54074401^,$26,$17.68,$0.52,$25.48,2019-11-26
3,YR7220,New York,Electronics,40901794,$43,$31.39,$3.87,$39.13,2019-11-26
4,YR7220,New York%,Electronics,37012634H,$71,$46.15,$4.97,$66.03,2019-11-26


In [16]:
#store location temizleme

df2=df.withColumn("STORE_LOCATION",
                  regexp_replace("STORE_LOCATION",r'[^a-zA-Z\s]',''))

In [17]:
df2 = df2.withColumn("STORE_LOCATION",F.trim("STORE_LOCATION"))

In [18]:
df2.show()

+--------+--------------+----------------+----------+---+------+--------+------+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|    CP|DISCOUNT|    SP|      Date|
+--------+--------------+----------------+----------+---+------+--------+------+----------+
|  YR7220|      New York|     Electronics|  12254943|$31|$20.77|   $1.86|$29.14|2019-11-26|
|  YR7220|      New York|       Furniture| 72619323C|$15| $9.75|    $1.5| $13.5|2019-11-26|
|  YR7220|      New York|     Electronics| 34161682B|$88|$62.48|    $4.4| $83.6|2019-11-26|
|  YR7220|      New York|         Kitchen|  79411621|$91|$58.24|   $3.64|$87.36|2019-11-26|
|  YR7220|      New York|         Fashion| 39520263T|$85|   $51|   $2.55|$82.45|2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204|$37|$24.05|   $0.74|$36.26|2019-11-26|
|  YR7220|      New York|       Cosmetics| 86610412D|$80| $48.8|    $6.4| $73.6|2019-11-26|
|  YR7220|      New York|         Kitchen| 52503356^|$71| $42.6|   $5.68|$65.32|

In [19]:
#PRODUCT_ID temizleme ve donusturme

#replace
df2=df2.withColumn("PRODUCT_ID",regexp_replace("PRODUCT_ID",r'[^0-9]',''))
#trim
df2=df2.withColumn("PRODUCT_ID",F.trim("PRODUCT_ID"))
#convert int
df2=df2.withColumn("PRODUCT_ID",F.col("PRODUCT_ID").cast("int"))

df2.dtypes


[('STORE_ID', 'string'),
 ('STORE_LOCATION', 'string'),
 ('PRODUCT_CATEGORY', 'string'),
 ('PRODUCT_ID', 'int'),
 ('MRP', 'string'),
 ('CP', 'string'),
 ('DISCOUNT', 'string'),
 ('SP', 'string'),
 ('Date', 'date')]

In [20]:
df2.show()

+--------+--------------+----------------+----------+---+------+--------+------+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|    CP|DISCOUNT|    SP|      Date|
+--------+--------------+----------------+----------+---+------+--------+------+----------+
|  YR7220|      New York|     Electronics|  12254943|$31|$20.77|   $1.86|$29.14|2019-11-26|
|  YR7220|      New York|       Furniture|  72619323|$15| $9.75|    $1.5| $13.5|2019-11-26|
|  YR7220|      New York|     Electronics|  34161682|$88|$62.48|    $4.4| $83.6|2019-11-26|
|  YR7220|      New York|         Kitchen|  79411621|$91|$58.24|   $3.64|$87.36|2019-11-26|
|  YR7220|      New York|         Fashion|  39520263|$85|   $51|   $2.55|$82.45|2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204|$37|$24.05|   $0.74|$36.26|2019-11-26|
|  YR7220|      New York|       Cosmetics|  86610412|$80| $48.8|    $6.4| $73.6|2019-11-26|
|  YR7220|      New York|         Kitchen|  52503356|$71| $42.6|   $5.68|$65.32|

In [21]:
#MRP temizleme ve donusturme

#replace
df2=df2.withColumn("MRP",regexp_replace("MRP",r'[^0-9]',''))
#trim
df2=df2.withColumn("MRP",F.trim("MRP"))
#
df2=df2.withColumn("MRP",F.col("MRP").cast("int"))


df2.dtypes


[('STORE_ID', 'string'),
 ('STORE_LOCATION', 'string'),
 ('PRODUCT_CATEGORY', 'string'),
 ('PRODUCT_ID', 'int'),
 ('MRP', 'int'),
 ('CP', 'string'),
 ('DISCOUNT', 'string'),
 ('SP', 'string'),
 ('Date', 'date')]

In [22]:
df2.show()

+--------+--------------+----------------+----------+---+------+--------+------+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|    CP|DISCOUNT|    SP|      Date|
+--------+--------------+----------------+----------+---+------+--------+------+----------+
|  YR7220|      New York|     Electronics|  12254943| 31|$20.77|   $1.86|$29.14|2019-11-26|
|  YR7220|      New York|       Furniture|  72619323| 15| $9.75|    $1.5| $13.5|2019-11-26|
|  YR7220|      New York|     Electronics|  34161682| 88|$62.48|    $4.4| $83.6|2019-11-26|
|  YR7220|      New York|         Kitchen|  79411621| 91|$58.24|   $3.64|$87.36|2019-11-26|
|  YR7220|      New York|         Fashion|  39520263| 85|   $51|   $2.55|$82.45|2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204| 37|$24.05|   $0.74|$36.26|2019-11-26|
|  YR7220|      New York|       Cosmetics|  86610412| 80| $48.8|    $6.4| $73.6|2019-11-26|
|  YR7220|      New York|         Kitchen|  52503356| 71| $42.6|   $5.68|$65.32|

In [23]:
#CP temizleme ve donusturme
df2=df2.withColumn("CP",regexp_replace("CP",r'[^0-9]',''))
df2=df2.withColumn("CP",F.trim("CP"))
df2=df2.withColumn("CP",F.col("CP").cast("int"))

df2.dtypes


[('STORE_ID', 'string'),
 ('STORE_LOCATION', 'string'),
 ('PRODUCT_CATEGORY', 'string'),
 ('PRODUCT_ID', 'int'),
 ('MRP', 'int'),
 ('CP', 'int'),
 ('DISCOUNT', 'string'),
 ('SP', 'string'),
 ('Date', 'date')]

In [24]:
df2.show()

+--------+--------------+----------------+----------+---+----+--------+------+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|  CP|DISCOUNT|    SP|      Date|
+--------+--------------+----------------+----------+---+----+--------+------+----------+
|  YR7220|      New York|     Electronics|  12254943| 31|2077|   $1.86|$29.14|2019-11-26|
|  YR7220|      New York|       Furniture|  72619323| 15| 975|    $1.5| $13.5|2019-11-26|
|  YR7220|      New York|     Electronics|  34161682| 88|6248|    $4.4| $83.6|2019-11-26|
|  YR7220|      New York|         Kitchen|  79411621| 91|5824|   $3.64|$87.36|2019-11-26|
|  YR7220|      New York|         Fashion|  39520263| 85|  51|   $2.55|$82.45|2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204| 37|2405|   $0.74|$36.26|2019-11-26|
|  YR7220|      New York|       Cosmetics|  86610412| 80| 488|    $6.4| $73.6|2019-11-26|
|  YR7220|      New York|         Kitchen|  52503356| 71| 426|   $5.68|$65.32|2019-11-26|
|  YR7220|

In [25]:
#DISCOUNT temizleme ve donusturme
df2=df2.withColumn("DISCOUNT",regexp_replace("DISCOUNT",r'[^0-9]',''))
df2=df2.withColumn("DISCOUNT",F.trim("DISCOUNT"))
df2=df2.withColumn("DISCOUNT",F.col("DISCOUNT").cast("int"))

df2.dtypes

[('STORE_ID', 'string'),
 ('STORE_LOCATION', 'string'),
 ('PRODUCT_CATEGORY', 'string'),
 ('PRODUCT_ID', 'int'),
 ('MRP', 'int'),
 ('CP', 'int'),
 ('DISCOUNT', 'int'),
 ('SP', 'string'),
 ('Date', 'date')]

In [26]:
df2.show()

+--------+--------------+----------------+----------+---+----+--------+------+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|  CP|DISCOUNT|    SP|      Date|
+--------+--------------+----------------+----------+---+----+--------+------+----------+
|  YR7220|      New York|     Electronics|  12254943| 31|2077|     186|$29.14|2019-11-26|
|  YR7220|      New York|       Furniture|  72619323| 15| 975|      15| $13.5|2019-11-26|
|  YR7220|      New York|     Electronics|  34161682| 88|6248|      44| $83.6|2019-11-26|
|  YR7220|      New York|         Kitchen|  79411621| 91|5824|     364|$87.36|2019-11-26|
|  YR7220|      New York|         Fashion|  39520263| 85|  51|     255|$82.45|2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204| 37|2405|      74|$36.26|2019-11-26|
|  YR7220|      New York|       Cosmetics|  86610412| 80| 488|      64| $73.6|2019-11-26|
|  YR7220|      New York|         Kitchen|  52503356| 71| 426|     568|$65.32|2019-11-26|
|  YR7220|

In [27]:
# SP temizleme ve donusturme
df2=df2.withColumn("SP",regexp_replace("SP",r'[^0-9]',''))
df2=df2.withColumn("SP",F.trim("SP"))
df2=df2.withColumn("SP",F.col("SP").cast("int"))

df2.dtypes

[('STORE_ID', 'string'),
 ('STORE_LOCATION', 'string'),
 ('PRODUCT_CATEGORY', 'string'),
 ('PRODUCT_ID', 'int'),
 ('MRP', 'int'),
 ('CP', 'int'),
 ('DISCOUNT', 'int'),
 ('SP', 'int'),
 ('Date', 'date')]

In [28]:
df2.show()

+--------+--------------+----------------+----------+---+----+--------+----+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|  CP|DISCOUNT|  SP|      Date|
+--------+--------------+----------------+----------+---+----+--------+----+----------+
|  YR7220|      New York|     Electronics|  12254943| 31|2077|     186|2914|2019-11-26|
|  YR7220|      New York|       Furniture|  72619323| 15| 975|      15| 135|2019-11-26|
|  YR7220|      New York|     Electronics|  34161682| 88|6248|      44| 836|2019-11-26|
|  YR7220|      New York|         Kitchen|  79411621| 91|5824|     364|8736|2019-11-26|
|  YR7220|      New York|         Fashion|  39520263| 85|  51|     255|8245|2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204| 37|2405|      74|3626|2019-11-26|
|  YR7220|      New York|       Cosmetics|  86610412| 80| 488|      64| 736|2019-11-26|
|  YR7220|      New York|         Kitchen|  52503356| 71| 426|     568|6532|2019-11-26|
|  YR7220|      New York|       

In [30]:
#postgresql
config = configparser.RawConfigParser()

config.read('./db_conn')
user_name = config.get('DB', 'user_name')
password = config.get('DB', 'password')
db_ip = config.get('DB', 'db_ip')

In [31]:
db_ip

'localhost'

In [32]:
jdbcUrl = f"jdbc:postgresql://{db_ip}:5433/odev_spark?user={user_name}&password={password}"

In [33]:
df2.write.jdbc(url=jdbcUrl,
              table="odev_clean_table", 
              mode="overwrite", 
              properties={"driver": 'org.postgresql.Driver'})